In [33]:
import torch
import numpy as np
import re
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

class FactVerificationScore:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        self.model = BertModel.from_pretrained('bert-base-cased')
        self.model.eval()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

    def _encode_text(self, text):
        inputs = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**{k: v.to(self.device) for k, v in inputs.items()})
            embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        return embeddings

    def _extract_facts(self, text):
        """ Извлекает числа, даты, ключевые сущности, включая сокращения """
        return set(re.findall(r'\b(?:НИУ )?ВШЭ|\b\d{4}\b|[А-ЯA-Z][а-яa-z]+(?: [А-ЯA-Z][а-яa-z]+)*', text))

    def calculate_score(self, model_output, context):
        # 1. Концептуальная согласованность
        model_embeddings = self._encode_text(model_output)
        context_embeddings = self._encode_text(context)
        concept_score = cosine_similarity(model_embeddings, context_embeddings)[0][0]

        # 2. Проверка фактологической точности
        model_facts = self._extract_facts(model_output)
        context_facts = self._extract_facts(context)

        correct_facts = len(model_facts & context_facts)
        hallucinated_facts = len(model_facts - context_facts)
        missing_facts = len(context_facts - model_facts)

        if len(model_facts) == 0:
            fact_error_ratio = 1 if len(context_facts) > 0 else 0
        else:
            fact_error_ratio = (hallucinated_facts + missing_facts) / len(model_facts)

        # 3. Итоговая метрика: ближе к 0 - ответ корректный, ближе к 1 - галлюцинация
        w_concept = 0.5
        w_fact_error = 0.5  # Усиленный штраф за фактологические ошибки

        hallucination_score = (w_concept * (1 - concept_score)) + (w_fact_error * fact_error_ratio)
        hallucination_score = min(1, max(0, hallucination_score))  # Ограничение в пределах [0, 1]

        return {
            "hallucination_score": hallucination_score,
            "concept_score": concept_score,
            "fact_error_ratio": fact_error_ratio,
            "hallucinated_facts": hallucinated_facts,
            "missing_facts": missing_facts
        }

In [36]:
# Пример использования
if __name__ == "__main__":
    evaluator = FactVerificationScore()

    model_output_1 = "Если студент не согласен с оценкой, он может подать апелляцию, но не пересдавать экзамен."
    context_1 = "Пересдача возможна только в случае академической задолженности."

    # А здесь покажет 0, так как всё очень корректно
    context_2 = "Пересдача возможна только в случае академической задолженности."
    model_output_2 = "Пересдача возможна только в случае академической задолженности."

    result_1 = evaluator.calculate_score(model_output_1, context_1)
    print("Тестирование №1:")
    print(f"Метрика галлюцинации: {result_1['hallucination_score']}")
    print(f"Концептуальная согласованность: {result_1['concept_score']}")
    print(f"Доля ошибок в фактах: {result_1['fact_error_ratio']}")
    print(f"Количество галлюцинированных фактов: {result_1['hallucinated_facts']}")
    print(f"Количество пропущенных фактов: {result_1['missing_facts']}")
    print("\n")

    result_2 = evaluator.calculate_score(model_output_2, context_2)
    print("Тестирование №2:")
    print(f"Метрика галлюцинации: {result_2['hallucination_score']}")
    print(f"Концептуальная согласованность: {result_2['concept_score']}")
    print(f"Доля ошибок в фактах: {result_2['fact_error_ratio']}")
    print(f"Количество галлюцинированных фактов: {result_2['hallucinated_facts']}")
    print(f"Количество пропущенных фактов: {result_2['missing_facts']}")

Тестирование №1:
Метрика галлюцинации: 1
Концептуальная согласованность: 0.9799569845199585
Доля ошибок в фактах: 2.0
Количество галлюцинированных фактов: 1
Количество пропущенных фактов: 1


Тестирование №2:
Метрика галлюцинации: 0
Концептуальная согласованность: 1.0
Доля ошибок в фактах: 0.0
Количество галлюцинированных фактов: 0
Количество пропущенных фактов: 0


Математическая модель метрики галлюцинации, реализованная в коде, выглядит следующим образом:

# Обозначения:

- $S_{h}$ — итоговый **hallucination score** (метрика галлюцинации).
- $S_c$ — **concept score** (концептуальная согласованность), вычисляемая через **косинусное сходство** между эмбеддингами ответа модели и контекста.
- $F_h$ — количество **галлюцинированных фактов** (факты в ответе модели, которых нет в контексте).
- $F_m$ — количество **пропущенных фактов** (факты, присутствующие в контексте, но отсутствующие в ответе модели).
- $F_{total}$ — общее число фактов в ответе модели.
- $w_c$ = 0.4 — вес концептуальной согласованности.
- $w_f$ = 0.6 — вес фактологической ошибки.
---

## 1. Концептуальная согласованность

$
S_c = \cos(\theta) = \frac{V_{model} \cdot V_{context}}{|V_{model}| |V_{context}|}
$

где \( $V_{model}$ \) и \( $V_{context}$ \) — это BERT-эмбеддинги CLS-токенов.

---

## 2. Доля фактологических ошибок
$
E_f =
\begin{cases}
1, & \text{если } F_{total} = 0 \text{ и } |F_m| > 0 \\
0, & \text{если } F_{total} = 0 \text{ и } |F_m| = 0 \\
\frac{|F_h| + |F_m|}{F_{total}}, & \text{иначе}
\end{cases}$

где $E_f$ — **fact error ratio** (доля ошибок в фактах).

---

## 3. Итоговая метрика галлюцинации
$
S_h = w_c (1 - S_c) + w_f E_f
$

где:

- Чем выше $S_h$, тем больше модель галлюцинирует.
- Чем ниже $S_h$, тем качественнее ответ модели.
- $S_h$ нормализуется в пределах $[0, 1]$:

$
S_h = \min(1, \max(0, S_h))
$
